In [365]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [366]:
train=pd.read_csv('../Data/train.csv')
test=pd.read_csv('../Data/test.csv')

In [367]:
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 500)

In [368]:
def treat(tmp):
    if tmp!=tmp:
        return False
    else:
        return True
def plotHeatMap(df, size=(16,9),an=True):
    plt.figure(figsize=size)
    cmap = plt.cm.viridis
    sns.heatmap(df.corr(),cmap=cmap,linewidths=0.2,vmax=1, linecolor="white", square=True,annot=an)
    plt.show()
    
def treat_cat(tmp):
    if tmp!=tmp:
        return 'unknown'
    else:
        return tmp
    
def max_min(data):
    data=pd.DataFrame(data)
    print('max = '+str(max(data.values.ravel()))+'  min = '+str(min(data.values.ravel())))
    

def data_type(data):
    data=pd.DataFrame(data)
    for i in data.values.ravel():
        if i==i:
            return type(i)
def non_zero(data):
    data=pd.DataFrame(data)
    print('non zero values = '+str(len(data[data.values.ravel()!=0]))+
         '   zero values = '+str(len(data[data.values.ravel()==0])))

def drop(data=None,col=None):
    return data.drop([i for i in col],axis=1)
    
def null_values(data):
     print('null values = '+str(len(data[data!=data])),' (',(len(data[data!=data])/len(data))*100,'%)')
        
def treat_num(x,m):
    if x!=x:
        return m
    else:
        return x
    
def show_percentile(x=None,data=None,lb=0,ub=100):
    lb,ub=np.percentile(data[x],q=(lb,ub),axis=0)
    print('sacrifice : ',len(data[x])-len(data[(data[x]>=lb) &(data[x]<=ub)]))
    describe(data[x][(data[x]>=lb) &(data[x]<=ub)])
    
def apply_percentile(x=None,data=None,lb=0,ub=100):
    lb,ub=np.percentile(data[x],q=(lb,ub),axis=0)
    return data[(data[x]>=lb) &(data[x]<=ub)]

def count(data):
    from collections import Counter
    print(Counter(data))
    
def consider_unknown(data,li):
    if data in li:
        return data
    else :
        return 'unknown'
    
def parse_date(dat):
    return str(dat[:4]+'-'+dat[4:6]+'-'+dat[6:])

def return_parts_of_date(tmp,sep='-'):
#     tmp=2004-04-20
    li=[]
    ptr,lst='',0
    for i in tmp:
        lst+=1
        if i!=sep:
            ptr+=i
        else :
            li.append(ptr)
            ptr=''
        if lst==len(tmp):
            li.append(ptr)
    return int(li[0]),int(li[1]),int(li[2])

def date_diff_in_days(date1=None,from_date2=None):
#     date dataframe must be in yyyy-mm-dd format
    date1,from_date2=pd.DataFrame(date1),pd.DataFrame(from_date2)
    from datetime import date
    li=[]
    for i,j in zip(date1.values,from_date2.values):
        y1,m1,d1=return_parts_of_date(str(i[0]))
        y2,m2,d2=return_parts_of_date(str(j[0]))
        dte1,dte2=date(y1,m1,d1),date(y2,m2,d2)
        delta=dte2-dte1
        li.append(delta.days)
    return li
    

# Outlier detection and worthless coulmns removal (cleaning)

In [369]:
#remove desk id as contain more than 30 % missing values
# train=drop(data=train,col=['desk_id'])
# test=drop(data=test,col=['desk_id'])
# stop , let's give it a shot by categorising these desk_id as unknown and label encoding them
train['is_desk_id_missing']=[1 if i!=i else 0 for i in train['desk_id'].values.ravel()]
test['is_desk_id_missing']=[1 if i!=i else 0 for i in test['desk_id'].values.ravel()]

train=drop(col=['desk_id'],data=train)
test=drop(col=['desk_id'],data=test)

In [370]:
# found 2 sold vaues in train (not in test :)are missing, so remove it
# these same 2 values are nan for bought, so no need to treat it separately, both are treated simultaneously
print(train.shape)
train=train[train['sold']==train['sold']]
print(train.shape)

(9366, 18)
(9364, 18)


In [371]:
# in sold , found minimum value too less, like outlier
show_percentile(x='sold',data=train,lb=2,ub=100)
show_percentile(x='sold',data=test,lb=2,ub=100)
#well, decision is to replace outlier with below minimum value to prevent data and supress the effect
train['sold']=[2112000.0 if i<2112000.0 else i for i in train['sold'].values]
test['sold']=[1969000.0 if i<1969000.0 else i for i in test['sold'].values]

sacrifice :  187
null values = 0  ( 0.0 %)
max = 13200000000.0  min = 2112000.0
non zero values = 9177   zero values = 0
               sold
count  9.177000e+03
mean   1.040372e+08
std    1.942132e+08
min    2.112000e+06
25%    3.095400e+07
50%    5.732100e+07
75%    1.173480e+08
max    1.320000e+10 

sacrifice :  96
null values = 0  ( 0.0 %)
max = 1560212500.0  min = 1969000.0
non zero values = 4705   zero values = 0
               sold
count  4.705000e+03
mean   1.045697e+08
std    1.361945e+08
min    1.969000e+06
25%    3.190000e+07
50%    5.957600e+07
75%    1.210000e+08
max    1.560212e+09 



In [372]:
# in pf_category , found category 'E' as only 2 counts (only in test data) , so minor outlier, remove these 2 entries
print(train.shape)
train=train[train['pf_category']!='E']
print(train.shape)

(9364, 18)
(9362, 18)


In [373]:
# found some 465 null values (5%) in both training and testing set for libor_rate,replacing by mean
train['libor_rate'][train['libor_rate']!=train['libor_rate']]=0.998066
test['libor_rate'][test['libor_rate']!=test['libor_rate']]=0.930464

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [374]:
show_percentile(x='bought',data=train,lb=1,ub=100)
show_percentile(x='bought',data=test,lb=.92,ub=100)
#found minimum values of bought unexpected as outlier in both train and test, so replace it will min after percentile
train['bought'][train['bought']<659779.798491]=659779.798491
test['bought'][test['bought']<659738.670184]=659738.670184

sacrifice :  94
null values = 0  ( 0.0 %)
max = 13199999266.7  min = 659779.798491
non zero values = 9268   zero values = 0
             bought
count  9.268000e+03
mean   1.029147e+08
std    1.934687e+08
min    6.597798e+05
25%    2.896266e+07
50%    5.637100e+07
75%    1.154857e+08
max    1.320000e+10 

sacrifice :  45
null values = 0  ( 0.0 %)
max = 1560167861.86  min = 659738.670184
non zero values = 4756   zero values = 0
             bought
count  4.756000e+03
mean   1.033844e+08
std    1.357927e+08
min    6.597387e+05
25%    3.040387e+07
50%    5.830416e+07
75%    1.209925e+08
max    1.560168e+09 



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [375]:
#Treating Nan's for indicator_code :

train['indicator_code']=train['indicator_code'].apply(lambda x : treat(x))
test['indicator_code']=test['indicator_code'].apply(lambda x : treat(x))

In [376]:
# for type , G not in test, so removing from train
print(train.shape)
train=train[train['type']!='G']
print(train.shape)

(9362, 18)
(9361, 18)


In [377]:
#found so many (60%) missing values, so categorising them to unknown category
train['hedge_value']=train['hedge_value'].apply(lambda y:treat_cat(tmp=y))
test['hedge_value']=test['hedge_value'].apply(lambda y:treat_cat(tmp=y))

In [378]:
#found so many (33%) missing values, so categorising them to unknown category
train['status']=train['status'].apply(lambda y:treat(tmp=y))
test['status']=test['status'].apply(lambda y:treat(tmp=y))

In [379]:
# len(train[train['bought']!=train['bought']]),len(test[test['bought']!=test['bought']])

# Feature extraction from data

In [380]:
#start date , creation_date ,sell_date 
#parsing date to pandas date format
train['creation_date']=train['creation_date'].apply(lambda x:parse_date(str(x)))
train['start_date']=train['start_date'].apply(lambda x:parse_date(str(x)))
train['sell_date']=train['sell_date'].apply(lambda x:parse_date(str(x)))

test['creation_date']=test['creation_date'].apply(lambda x:parse_date(str(x)))
test['start_date']=test['start_date'].apply(lambda x:parse_date(str(x)))
test['sell_date']=test['sell_date'].apply(lambda x:parse_date(str(x)))

In [381]:
#creating features between two dates to measure time interval between various works
train['create-start_time']=date_diff_in_days(date1=train['creation_date'],from_date2=train['start_date'])
train['sell-start_time']=date_diff_in_days(date1=train['sell_date'],from_date2=train['start_date'])
train['sell-create_time']=date_diff_in_days(date1=train['sell_date'],from_date2=train['creation_date'])

test['create-start_time']=date_diff_in_days(date1=test['creation_date'],from_date2=test['start_date'])
test['sell-start_time']=date_diff_in_days(date1=test['sell_date'],from_date2=test['start_date'])
test['sell-create_time']=date_diff_in_days(date1=test['sell_date'],from_date2=test['creation_date'])

In [382]:
# describe(data=train['sell-create_time'],with_plot=True)

In [383]:
#converting parsed date to datetime
train['creation_date']=pd.to_datetime(train['creation_date'])
train['start_date']=pd.to_datetime(train['start_date'])
train['sell_date']=pd.to_datetime(train['sell_date'])

test['creation_date']=pd.to_datetime(test['creation_date'])
test['start_date']=pd.to_datetime(test['start_date'])
test['sell_date']=pd.to_datetime(test['sell_date'])

In [384]:
#feature extraction from datetime
train['creation_month']=train.creation_date.dt.month
train['creation_year']=train.creation_date.dt.year
train['creation_day']=train.creation_date.dt.day

train['start_month']=train.start_date.dt.month
train['start_year']=train.start_date.dt.year
train['start_day']=train.start_date.dt.day

train['sell_month']=train.sell_date.dt.month
train['sell_year']=train.sell_date.dt.year
train['sell_day']=train.sell_date.dt.day

test['creation_month']=test.creation_date.dt.month
test['creation_year']=test.creation_date.dt.year
test['creation_day']=test.creation_date.dt.day

test['start_month']=test.start_date.dt.month
test['start_year']=test.start_date.dt.year
test['start_day']=test.start_date.dt.day

test['sell_month']=test.sell_date.dt.month
test['sell_year']=test.sell_date.dt.year
test['sell_day']=test.sell_date.dt.day

In [385]:
#Transforming cyclic features

# cyclic_feature=['sell_day','sell_month','creation_day','creation_month','start_day','start_month']

# for i in cyclic_feature:
#     train[i]=abs(np.sin(train[i].values))
#     test[i]=abs(np.sin(test[i].values))

In [386]:
train['sell_is_month_end']=train.sell_date.dt.is_month_end
train['sell_is_month_start']=train.sell_date.dt.is_month_start
train['sell_is_quarter_end']=train.sell_date.dt.is_quarter_end
train['sell_is_quarter_start']=train.sell_date.dt.is_quarter_start
train['sell_is_year_start']=train.sell_date.dt.is_year_start
train['sell_is_year_end']=train.sell_date.dt.is_year_end
train['sell_week_of_year']=train.sell_date.dt.weekofyear
train['sell_week_day']=train.sell_date.dt.weekday

test['sell_is_month_end']=test.sell_date.dt.is_month_end
test['sell_is_month_start']=test.sell_date.dt.is_month_start
test['sell_is_quarter_end']=test.sell_date.dt.is_quarter_end
test['sell_is_quarter_start']=test.sell_date.dt.is_quarter_start
test['sell_is_year_start']=test.sell_date.dt.is_year_start
test['sell_is_year_end']=test.sell_date.dt.is_year_end
test['sell_week_of_year']=test.sell_date.dt.weekofyear
test['sell_week_day']=test.sell_date.dt.weekday

In [387]:
train['start_is_month_end']=train.start_date.dt.is_month_end
train['start_is_month_start']=train.start_date.dt.is_month_start
train['start_is_quarter_end']=train.start_date.dt.is_quarter_end
train['start_is_quarter_start']=train.start_date.dt.is_quarter_start
train['start_is_year_start']=train.start_date.dt.is_year_start
train['start_is_year_end']=train.start_date.dt.is_year_end
train['start_week_of_year']=train.start_date.dt.weekofyear
train['start_week_day']=train.start_date.dt.weekday

test['start_is_month_end']=test.start_date.dt.is_month_end
test['start_is_month_start']=test.start_date.dt.is_month_start
test['start_is_quarter_end']=test.start_date.dt.is_quarter_end
test['start_is_quarter_start']=test.start_date.dt.is_quarter_start
test['start_is_year_start']=test.start_date.dt.is_year_start
test['start_is_year_end']=test.start_date.dt.is_year_end
test['start_week_of_year']=test.start_date.dt.weekofyear
test['start_week_day']=test.start_date.dt.weekday

In [388]:
train['creation_is_month_end']=train.creation_date.dt.is_month_end
train['creation_is_month_start']=train.creation_date.dt.is_month_start
train['creation_is_quarter_end']=train.creation_date.dt.is_quarter_end
train['creation_is_quarter_start']=train.creation_date.dt.is_quarter_start
train['creation_is_year_start']=train.creation_date.dt.is_year_start
train['creation_is_year_end']=train.creation_date.dt.is_year_end
train['creation_week_of_year']=train.creation_date.dt.weekofyear
train['creation_week_day']=train.creation_date.dt.weekday

test['creation_is_month_end']=test.creation_date.dt.is_month_end
test['creation_is_month_start']=test.creation_date.dt.is_month_start
test['creation_is_quarter_end']=test.creation_date.dt.is_quarter_end
test['creation_is_quarter_start']=test.creation_date.dt.is_quarter_start
test['creation_is_year_start']=test.creation_date.dt.is_year_start
test['creation_is_year_end']=test.creation_date.dt.is_year_end
test['creation_week_of_year']=test.creation_date.dt.weekofyear
test['creation_week_day']=test.creation_date.dt.weekday

In [389]:
# train['creation_week_day']=abs(np.sin(train['creation_week_day'].values))
# test['creation_week_day']=abs(np.sin(test['creation_week_day'].values))

# train['sell_week_day']=abs(np.sin(train['sell_week_day'].values))
# test['sell_week_day']=abs(np.sin(test['sell_week_day'].values))

# train['start_week_day']=abs(np.sin(train['start_week_day'].values))
# test['start_week_day']=abs(np.sin(test['start_week_day'].values))

In [390]:
train=drop(col=['creation_date','sell_date','start_date'],data=train)
test=drop(col=['creation_date','sell_date','start_date'],data=test)

In [391]:
#negative euribor rate
train['is_negative_euribor_rate']=[1 if i<0 else 0 for i in train['euribor_rate'].values.ravel()]
test['is_negative_euribor_rate']=[1 if i<0 else 0 for i in test['euribor_rate'].values.ravel()]

In [392]:
# currency matters most, i think , so One hot encoding these features
cur_tr=pd.get_dummies(data=train['currency'],columns=['currency'])
cur_ts=pd.get_dummies(data=test['currency'],columns=['currency'])

In [393]:
# applying OHE to train
train=drop(col=['currency'],data=train)
test=drop(col=['currency'],data=test)

In [394]:
# applying OHE to train
train=pd.concat([train,cur_tr],axis=1)
test=pd.concat([test,cur_ts],axis=1)

In [395]:
#negative libor_rate
train['is_negative_libor_rate_rate']=[1 if i<0 else 0 for i in train['libor_rate'].values.ravel()]
test['is_negative_libor_rate_rate']=[1 if i<0 else 0 for i in test['libor_rate'].values.ravel()]

In [396]:
#is hedge_value missing
train['is_hedge_missing']=[1 if i=='unknown' else 0 for i in train['hedge_value'].values.ravel()]
test['is_hedge_missing']=[1 if i=='unknown' else 0 for i in test['hedge_value'].values.ravel()]

In [397]:
train.shape,test.shape

((9361, 58), (4801, 57))

# Encoding

In [398]:
train['hedge_value']=train.hedge_value.astype(str)
test['hedge_value']=test.hedge_value.astype(str)

train['indicator_code']=train.indicator_code.astype(str)
test['indicator_code']=test.indicator_code.astype(str)

train['status']=train.status.astype(str)
test['status']=test.status.astype(str)




train['sell_is_month_end']=train.sell_is_month_end.astype(str)
test['sell_is_month_end']=test.sell_is_month_end.astype(str)

train['sell_is_month_start']=train.sell_is_month_start.astype(str)
test['sell_is_month_start']=test.sell_is_month_start.astype(str)

train['sell_is_quarter_end']=train.sell_is_quarter_end.astype(str)
test['sell_is_quarter_end']=test.sell_is_quarter_end.astype(str)

train['sell_is_quarter_start']=train.sell_is_quarter_start.astype(str)
test['sell_is_quarter_start']=test.sell_is_quarter_start.astype(str)

train['sell_is_year_start']=train.sell_is_year_start.astype(str)
test['sell_is_year_start']=test.sell_is_year_start.astype(str)

train['sell_is_year_end']=train.sell_is_year_end.astype(str)
test['sell_is_year_end']=test.sell_is_year_end.astype(str)





train['start_is_month_end']=train.start_is_month_end.astype(str)
test['start_is_month_end']=test.start_is_month_end.astype(str)

train['start_is_month_start']=train.start_is_month_start.astype(str)
test['start_is_month_start']=test.start_is_month_start.astype(str)

train['start_is_quarter_end']=train.start_is_quarter_end.astype(str)
test['start_is_quarter_end']=test.start_is_quarter_end.astype(str)

train['start_is_quarter_start']=train.start_is_quarter_start.astype(str)
test['start_is_quarter_start']=test.start_is_quarter_start.astype(str)

train['start_is_year_start']=train.start_is_year_start.astype(str)
test['start_is_year_start']=test.start_is_year_start.astype(str)

train['start_is_year_end']=train.start_is_year_end.astype(str)
test['start_is_year_end']=test.start_is_year_end.astype(str)





train['creation_is_month_end']=train.creation_is_month_end.astype(str)
test['creation_is_month_end']=test.creation_is_month_end.astype(str)

train['creation_is_month_start']=train.creation_is_month_start.astype(str)
test['creation_is_month_start']=test.creation_is_month_start.astype(str)

train['creation_is_quarter_end']=train.creation_is_quarter_end.astype(str)
test['creation_is_quarter_end']=test.creation_is_quarter_end.astype(str)

train['creation_is_quarter_start']=train.creation_is_quarter_start.astype(str)
test['creation_is_quarter_start']=test.creation_is_quarter_start.astype(str)

train['creation_is_year_start']=train.creation_is_year_start.astype(str)
test['creation_is_year_start']=test.creation_is_year_start.astype(str)

train['creation_is_year_end']=train.creation_is_year_end.astype(str)
test['creation_is_year_end']=test.creation_is_year_end.astype(str)

In [399]:
train.shape,test.shape

((9361, 58), (4801, 57))

In [400]:
set(train['pf_category'])
# pf_category,country_code,office_id,indicator_code,type,hedge_value,status,

{'A', 'B', 'C', 'D'}

In [401]:
from sklearn.preprocessing import LabelEncoder
for i in train.columns:
    if i not in ['return','portfolio_id','pf_category',
                 'country_code','office_id','indicator_code',
                 'type','hedge_value','status'] and data_type(train[i])==str:
        le=LabelEncoder()
        train[i]=le.fit_transform(train[i].values)
        test[i]=le.transform(test[i].values)

In [402]:
for i in ['pf_category','country_code','office_id','indicator_code',
          'type','hedge_value','status']:
    lt=len(set(train[i]))
    cur_tr=pd.get_dummies(data=train[i],columns=[i])
    cur_ts=pd.get_dummies(data=test[i],columns=[i])
    
    cur_tr=pd.DataFrame(data=cur_tr)
    cur_ts=pd.DataFrame(data=cur_ts)
    
    cur_tr.columns=[i+str(j) for j in range(lt)]
    cur_ts.columns=[i+str(j) for j in range(lt)]
    
    # applying OHE to train
    train=drop(col=[i],data=train)
    test=drop(col=[i],data=test)

    # applying OHE to train
    train=pd.concat([train,cur_tr],axis=1)
    test=pd.concat([test,cur_ts],axis=1)
    print(train.shape)

(9361, 61)
(9361, 65)
(9361, 66)
(9361, 67)
(9361, 73)
(9361, 75)
(9361, 76)


In [403]:
train.head()

,portfolio_id,sold,euribor_rate,libor_rate,bought,return,is_desk_id_missing,create-start_time,sell-start_time,sell-create_time,creation_month,creation_year,creation_day,start_month,start_year,start_day,sell_month,sell_year,sell_day,sell_is_month_end,sell_is_month_start,sell_is_quarter_end,sell_is_quarter_start,sell_is_year_start,sell_is_year_end,sell_week_of_year,sell_week_day,start_is_month_end,start_is_month_start,start_is_quarter_end,start_is_quarter_start,start_is_year_start,start_is_year_end,start_week_of_year,start_week_day,creation_is_month_end,creation_is_month_start,creation_is_quarter_end,creation_is_quarter_start,creation_is_year_start,creation_is_year_end,creation_week_of_year,creation_week_day,is_negative_euribor_rate,CHF,EUR,GBP,JPY,USD,is_negative_libor_rate_rate,is_hedge_missing,pf_category0,pf_category1,pf_category2,pf_category3,country_code0,country_code1,country_code2,country_code3,country_code4,office_id0,office_id1,indicator_code0,indicator_code1,type0,type1,type2,type3,type4,type5,type6,hedge_value0,hedge_value1,hedge_value2,status0,status1
0,PF00001002,110000000.0,0.02074,2.332216,1.098097e+08,0.02496,0,0,-23,-23,7,2004,20,7,2004,20,8,2004,12,0,0,0,0,0,0,33,3,0,0,0,0,0,0,30,1,0,0,0,0,0,0,30,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0
1,PF00001003,176671000.0,0.02074,5.269617,1.760084e+08,0.05496,0,-14,-34,-20,7,2004,23,7,2004,9,8,2004,12,0,0,0,0,0,0,33,3,0,0,0,0,0,0,28,4,0,0,0,0,0,0,30,4,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0
2,PF00001005,56474000.0,0.02074,2.332216,5.637953e+07,0.02496,0,0,-25,-25,7,2004,23,7,2004,23,8,2004,17,0,0,0,0,0,0,34,1,0,0,0,0,0,0,30,4,0,0,0,0,0,0,30,4,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
3,PF00001006,164813000.0,0.02074,2.332216,1.645088e+08,0.02496,0,-44,-34,10,7,2004,23,6,2004,9,7,2004,13,0,0,0,0,0,0,29,1,0,0,0,0,0,0,24,2,0,0,0,0,0,0,30,4,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
4,PF00001007,140800000.0,0.02074,2.332216,1.405402e+08,0.02496,0,-44,-34,10,7,2004,23,6,2004,9,7,2004,13,0,0,0,0,0,0,29,1,0,0,0,0,0,0,24,2,0,0,0,0,0,0,30,4,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0


In [404]:
test.head()

,portfolio_id,sold,euribor_rate,libor_rate,bought,is_desk_id_missing,create-start_time,sell-start_time,sell-create_time,creation_month,creation_year,creation_day,start_month,start_year,start_day,sell_month,sell_year,sell_day,sell_is_month_end,sell_is_month_start,sell_is_quarter_end,sell_is_quarter_start,sell_is_year_start,sell_is_year_end,sell_week_of_year,sell_week_day,start_is_month_end,start_is_month_start,start_is_quarter_end,start_is_quarter_start,start_is_year_start,start_is_year_end,start_week_of_year,start_week_day,creation_is_month_end,creation_is_month_start,creation_is_quarter_end,creation_is_quarter_start,creation_is_year_start,creation_is_year_end,creation_week_of_year,creation_week_day,is_negative_euribor_rate,CHF,EUR,GBP,JPY,USD,is_negative_libor_rate_rate,is_hedge_missing,pf_category0,pf_category1,pf_category2,pf_category3,country_code0,country_code1,country_code2,country_code3,country_code4,office_id0,office_id1,indicator_code0,indicator_code1,type0,type1,type2,type3,type4,type5,type6,hedge_value0,hedge_value1,hedge_value2,status0,status1
0,PF00001001,171831000.0,0.02074,2.332216,1.715337e+08,0,0,-23,-23,7,2004,20,7,2004,20,8,2004,12,0,0,0,0,0,0,33,3,0,0,0,0,0,0,30,1,0,0,0,0,0,0,30,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
1,PF00001004,56485000.0,0.02074,2.332216,5.638038e+07,0,0,4,4,7,2004,23,7,2004,23,7,2004,19,0,0,0,0,0,0,30,0,0,0,0,0,0,0,30,4,0,0,0,0,0,0,30,4,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0
2,PF00001009,41734000.0,0.02074,2.332216,4.166179e+07,0,-49,-32,17,7,2004,26,6,2004,7,7,2004,9,0,0,0,0,0,0,28,4,0,0,0,0,0,0,24,0,0,0,0,0,0,0,31,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
3,PF00001013,82951000.0,0.02074,2.332216,8.250936e+07,0,-10,-95,-85,7,2004,26,7,2004,16,10,2004,19,0,0,0,0,0,0,43,1,0,0,0,0,0,0,29,4,0,0,0,0,0,0,31,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
4,PF00001014,73293000.0,0.02074,2.332216,7.317055e+07,0,-13,-31,-18,7,2004,26,7,2004,13,8,2004,13,0,0,0,0,0,0,33,4,0,0,0,0,0,0,29,1,0,0,0,0,0,0,31,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0


In [405]:
for i in test.columns:
    if len(set(train[i]))<=10:
        count(train[i])
        count(test[i])
        print(i,'\n\n')

Counter({0: 5697, 1: 3664})
Counter({0: 2853, 1: 1948})
is_desk_id_missing 


Counter({0: 8956, 1: 405})
Counter({0: 4589, 1: 212})
sell_is_month_end 


Counter({0: 9185, 1: 176})
Counter({0: 4695, 1: 106})
sell_is_month_start 


Counter({0: 9207, 1: 154})
Counter({0: 4723, 1: 78})
sell_is_quarter_end 


Counter({0: 9324, 1: 37})
Counter({0: 4784, 1: 17})
sell_is_quarter_start 


Counter({0: 9361})
Counter({0: 4801})
sell_is_year_start 


Counter({0: 9343, 1: 18})
Counter({0: 4788, 1: 13})
sell_is_year_end 


Counter({4: 2176, 0: 2074, 2: 1792, 1: 1724, 3: 1595})
Counter({4: 1122, 0: 1044, 2: 917, 1: 901, 3: 817})
sell_week_day 


Counter({0: 9093, 1: 268})
Counter({0: 4640, 1: 161})
start_is_month_end 


Counter({0: 9193, 1: 168})
Counter({0: 4717, 1: 84})
start_is_month_start 


Counter({0: 9262, 1: 99})
Counter({0: 4749, 1: 52})
start_is_quarter_end 


Counter({0: 9323, 1: 38})
Counter({0: 4783, 1: 18})
start_is_quarter_start 


Counter({0: 9361})
Counter({0: 4801})
start_is_year_st

In [406]:
# found some 99% zero's columns, drop it
# for i in train.columns:
#     if 'is_year' in i:
#         train=drop(col=[i],data=train)
#         test=drop(col=[i],data=test)

In [407]:
train.shape,test.shape

((9361, 76), (4801, 75))

# Saving cleaned and Preprocessed data :)  ^_^

In [408]:
train.to_csv('../Data/cleaned_train_best.csv',index=False)
test.to_csv('../Data/cleaned_test_best.csv',index=False)